# RUTINAS para SUBIR FORECAST

* La demanda calculada de los algoritmos genera un archivo solicitudes:compra.csv
* Esta información debe recuperar los ID de 
    - fnd_product y los 
    - fnd_sotre para subirlo a 
    - **forecast_result**

In [42]:
# LIBRERIAS NECESARIAS 
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pyodbc
from dotenv import dotenv_values
import psycopg2 as pg2    # Conectores para Postgres
import getpass  # Para obtener el usuario del sistema operativo

# Mostrar el DataFrame resultante
import ace_tools_open as tools

import uuid  # Importar la librería uuid

# Evitar Mensajes Molestos
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category= FutureWarning)

# Liberias para Algoritmos
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import Holt

secrets = dotenv_values(".env")   # Connection String from .env
folder = secrets["FOLDER_DATOS"]


In [ ]:
# FUNCIONES

###----------------------------------------------------------------
#     DATOS
###----------------------------------------------------------------
def Open_Connection():
    secrets = dotenv_values(".env")   # Connection String from .env
    conn_str = f'DRIVER={secrets["DRIVER2"]};SERVER={secrets["SERVIDOR2"]};PORT={secrets["PUERTO2"]};DATABASE={secrets["BASE2"]};UID={secrets["USUARIO2"]};PWD={secrets["CONTRASENA2"]}'
    # print (conn_str) 
    try:    
        conn = pyodbc.connect(conn_str)
        return conn
    except:
        print('Error en la Conexión')
        return None

def Open_Conn_Postgres():
    secrets = dotenv_values(".env")   # Cargar credenciales desde .env    
    conn_str = f"dbname={secrets['BASE3']} user={secrets['USUARIO3']} password={secrets['CONTRASENA3']} host={secrets['SERVIDOR3']} port={secrets['PUERTO3']}"
    #print (conn_str)
    try:    
        conn = pg2.connect(conn_str)
        return conn
    except Exception as e:
        print(f'Error en la conexión: {e}')
        return None

def Open_Conn_Connexa():
    secrets = dotenv_values(".env")   # Cargar credenciales desde .env    
    conn_str = f"dbname={secrets['BASE4']} user={secrets['USUARIO4']} password={secrets['CONTRASENA4']} host={secrets['SERVIDOR4']} port={secrets['PUERTO4']}"
    try:    
        conn = pg2.connect(conn_str)
        return conn
    except Exception as e:
        print(f'Error en la conexión: {e}')
        return None

def Close_Connection(conn): 
    conn.close()
    return True

# Helper para generar identificadores únicos
def id_aleatorio():
    return str(uuid.uuid4())
def Close_Connection(conn): 
    conn.close()
    return True

def generar_datos(id_proveedor, etiqueta):
    secrets = dotenv_values(".env")   # Connection String from .env
    folder = secrets["FOLDER_DATOS"]
    
    #  Intento recuperar datos cacheados
    try:
        data = pd.read_csv(f'{folder}/{etiqueta}.csv')
        data['Codigo_Articulo']= data['Codigo_Articulo'].astype(int)
        data['Sucursal']= data['Sucursal'].astype(int)
        data['Fecha']= pd.to_datetime(data['Fecha'])

        articulos = pd.read_csv(f'{folder}/{etiqueta}_articulos.csv')
        #articulos.head()
        print(f"-> Datos Recuperados del CACHE: {id_proveedor}, Label: {etiqueta}")
        return data, articulos
    except:     
        print(f"-> Generando datos para ID: {id_proveedor}, Label: {etiqueta}")
        # Configuración de conexión
        conn = Open_Connection()
        
        # FILTRA solo PRODUCTOS HABILITADOS y Traer datos de STOCK y PENDIENTES desde PRODUCCIÓN
        query = f"""
        SELECT  A.[C_PROVEEDOR_PRIMARIO]
            ,S.[C_ARTICULO]
            ,S.[C_SUCU_EMPR]
            ,S.[Q_FACTOR_VENTA_ESP]
            ,S.[Q_FACTOR_VTA_SUCU]
            ,S.[M_OFERTA_SUCU]
            ,S.[M_HABILITADO_SUCU]
            ,A.M_BAJA
            ,S.[Q_VTA_DIA_ANT]
            ,S.[Q_VTA_ACUM]
            ,S.[Q_ULT_ING_STOCK]
            ,S.[Q_STOCK_A_ULT_ING]
            ,S.[Q_15DIASVTA_A_ULT_ING_STOCK]
            ,S.[Q_30DIASVTA_A_ULT_ING_STOCK]
            ,S.[Q_BULTOS_PENDIENTE_OC]
            ,S.[Q_PESO_PENDIENTE_OC]
            ,S.[Q_UNID_PESO_PEND_RECEP_TRANSF]
            ,S.[Q_UNID_PESO_VTA_MES_ACTUAL]
            ,S.[F_ULTIMA_VTA]
            ,S.[Q_VTA_ULTIMOS_15DIAS]
            ,S.[Q_VTA_ULTIMOS_30DIAS]
            ,S.[Q_TRANSF_PEND]
            ,S.[Q_TRANSF_EN_PREP]
            ,S.[M_FOLDER]
            ,S.[M_ALTA_RENTABILIDAD]
            ,S.[Lugar_Abastecimiento]
            ,S.[M_COSTO_LOGISTICO]
        
        FROM [DIARCOP001].[DiarcoP].[dbo].[T051_ARTICULOS_SUCURSAL] S
        LEFT JOIN [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] A
            ON A.[C_ARTICULO] = S.[C_ARTICULO]

        WHERE S.[M_HABILITADO_SUCU] = 'S' -- Permitido Reponer
            AND A.M_BAJA = 'N'  -- Activo en Maestro Artículos
            AND A.[C_PROVEEDOR_PRIMARIO] = {id_proveedor} -- Solo del Proveedor
        ;
        """
        # Ejecutar la consulta SQL
        articulos = pd.read_sql(query, conn)
        file_path = f'{folder}/{etiqueta}_articulos.csv'
        articulos['C_PROVEEDOR_PRIMARIO']= articulos['C_PROVEEDOR_PRIMARIO'].astype(int)
        articulos['C_ARTICULO']= articulos['C_ARTICULO'].astype(int)
        articulos.to_csv(file_path, index=False, encoding='utf-8')        
        print(f"---> Datos de Artículos guardados: {file_path}")
        
        # Consulta SQL para obtener las ventas de un proveedor específico   
        # Reemplazar {proveedor} en la consulta con el ID de la tienda actual
        query = f"""
        SELECT V.[F_VENTA] as Fecha
            ,V.[C_ARTICULO] as Codigo_Articulo
            ,V.[C_SUCU_EMPR] as Sucursal
            ,V.[I_PRECIO_VENTA] as Precio
            ,V.[I_PRECIO_COSTO] as Costo
            ,V.[Q_UNIDADES_VENDIDAS] as Unidades
            ,V.[C_FAMILIA] as Familia
            ,A.[C_RUBRO] as Rubro
            ,A.[C_SUBRUBRO_1] as SubRubro
            ,LTRIM(RTRIM(REPLACE(REPLACE(REPLACE(A.N_ARTICULO, CHAR(9), ''), CHAR(13), ''), CHAR(10), ''))) as Nombre_Articulo
            ,A.[C_CLASIFICACION_COMPRA] as Clasificacion
        FROM [DCO-DBCORE-P02].[DiarcoEst].[dbo].[T702_EST_VTAS_POR_ARTICULO] V
        LEFT JOIN [DCO-DBCORE-P02].[DiarcoEst].[dbo].[T050_ARTICULOS] A 
            ON V.C_ARTICULO = A.C_ARTICULO
        WHERE A.[C_PROVEEDOR_PRIMARIO] = {id_proveedor} AND V.F_VENTA >= '20210101' AND A.M_BAJA ='N'
        ORDER BY V.F_VENTA ;
        """

        # Ejecutar la consulta SQL
        demanda = pd.read_sql(query, conn)
        
        # UNIR Y FILTRAR solo la demanda de los Hartículos VALIDOS.
        # Realizar la unión (merge) de los DataFrames por las claves especificadas
        data = pd.merge(
            articulos,  # DataFrame de artículos
            demanda,    # DataFrame de demanda
            left_on=['C_ARTICULO', 'C_SUCU_EMPR'],  # Claves en 'articulos'
            right_on=['Codigo_Articulo', 'Sucursal'],  # Claves en 'demanda'
            how='inner'  # Solo traer los productos que están en 'articulos'
        )
            
        # Guardar los resultados en un archivo CSV con el nombre del Proveedor
        file_path = f'{folder}/{etiqueta}_FULL.csv'
        data['C_ARTICULO']= data['C_ARTICULO'].astype(int)
        data['C_SUCU_EMPR']= data['C_SUCU_EMPR'].astype(int)
        data.to_csv(file_path, index=False, encoding='utf-8')
        print(f"---> Datos de FULL guardados: {file_path}")        
        # Eliminar Columnas Innecesarias
        data = data[['Fecha', 'Codigo_Articulo', 'Sucursal', 'Unidades']]
        
        # Guardar los resultados en un archivo CSV con el nombre del Proveedor
        file_path = f'{folder}/{etiqueta}.csv'
        data.to_csv(file_path, index=False, encoding='utf-8')
        
        # Cerrar la conexión después de la iteración
        Close_Connection(conn)
    return data, articulos
    
def Exportar_Pronostico(df_forecast, proveedor, etiqueta, algoritmo):
    df_forecast['Codigo_Articulo']= df_forecast['Codigo_Articulo'].astype(int)
    df_forecast['Sucursal']= df_forecast['Sucursal'].astype(int)

    # tools.display_dataframe_to_user(name="SET de Datos del Proveedor", dataframe=df_forecast)
    # df_forecast.info()
    print(f'-> ** Pronostico Guardado en: {folder}/{etiqueta}_Pronostico_{algoritmo}.csv **')
    df_forecast.to_csv(f'{folder}/{etiqueta}_Pronostico_{algoritmo}.csv', index=False)

    ## GUARDAR TABLA EN POSTGRES
    usuario = getpass.getuser()  # Obtiene el usuario del sistema operativo
    fecha_actual = datetime.today().strftime('%Y-%m-%d')  # Obtiene la fecha de hoy en formato 'YYYY-MM-DD'

    conn = Open_Conn_Postgres()

    # Query de inserción
    insert_query = """
    INSERT INTO public.f_oc_precarga_connexa (
        c_proveedor, c_articulo, c_sucu_empr, q_forecast_unidades, f_alta_forecast, c_usuario_forecast, create_date
    ) VALUES (%s, %s, %s, %s, %s, %s, %s);
    """

    # Convertir el DataFrame a una lista de tuplas para la inserción en bloque
    data_to_insert = [
        (proveedor, row['Codigo_Articulo'], row['Sucursal'], row['Forecast'], fecha_actual, usuario, fecha_actual)
        for _, row in df_forecast.iterrows()
    ]

    try:
        with conn.cursor() as cur:
            cur.executemany(insert_query, data_to_insert)
        conn.commit()
        print(f"✅ Inserción completada: {len(data_to_insert)} registros insertados.")
    except Exception as e:
        conn.rollback()
        print(f"❌ Error en la inserción: {e}")
    finally:
        Close_Connection(conn)
        print("✅ Conexión cerrada.")
    return True     

def get_forecast_execution():
    print(f"-> Generando datos Forecast Excecution")
    # Configuración de conexión
    conn = Open_Conn_Connexa()
    
    # FILTRA solo LOS PROCESOS EJECUTADOS con su correspondientes SCHEDULES
    query = """
    SELECT fe.id, fe.description, fe.name, fe."timestamp", fe.supply_forecast_model_id,
		fee.id AS spl_supply_forecast_execution_execute_id
	FROM public.spl_supply_forecast_execution fe
	JOIN public.spl_supply_forecast_execution_execute fee
	ON fe.id = fee.supply_forecast_execution_id;	
    ;
    """
    # Ejecutar la consulta SQL
    forecast_exec = pd.read_sql(query, conn)
    file_path = f'{folder}/forecast_exec.csv'
    
    forecast_exec.to_csv(file_path, index=False, encoding='utf-8')      
    print(f"---> Datos de Forecast Execution guardados: {file_path}")
    conn.close()
    return forecast_exec

def get_fnd_product():
    print(f"-> Generando datos Productos Connexa")
    # Configuración de conexión
    conn = Open_Conn_Connexa()
    
    # FILTRA solo PRODUCTOS HABILITADOS y Traer datos de STOCK y PENDIENTES desde PRODUCCIÓN
    query = """
    SELECT id, description, ext_code,  sku, "timestamp"
	FROM public.fnd_product;
    """
    # Ejecutar la consulta SQL
    fnd_product = pd.read_sql(query, conn)
    file_path = f'{folder}/fnd_productc.csv'
    
    fnd_product.to_csv(file_path, index=False, encoding='utf-8')      
    print(f"---> Datos de PRODUCT en CONNEXA guardados: {file_path}")
    conn.close() 
    return fnd_product
    
    
def get_fnd_site():
    print(f"-> Generando datos Productos Connexa")
    # Configuración de conexión
    conn = Open_Conn_Connexa()
    
    # FILTRA solo PRODUCTOS HABILITADOS y Traer datos de STOCK y PENDIENTES desde PRODUCCIÓN
    query = """
    SELECT id, name, "timestamp", type, code, company_id
	FROM public.fnd_site;
    """
    # Ejecutar la consulta SQL
    fnd_site = pd.read_sql(query, conn)
    file_path = f'{folder}/fnd_site.csv'
    
    fnd_site.to_csv(file_path, index=False, encoding='utf-8')      
    print(f"---> Datos de STORES en CONNEXA guardados: {file_path}")
    conn.close()
    return fnd_site

In [4]:
def get_solicitudes(id_proveedor, etiqueta):
    secrets = dotenv_values(".env")   # Connection String from .env
    folder = secrets["FOLDER_DATOS"]
    
    #  Intento recuperar datos cacheados
    try:
        data = pd.read_csv(f'{folder}/{etiqueta}_Solicitudes_Compra.csv')
        data['Codigo_Articulo']= data['Codigo_Articulo'].astype(int)
        data['Sucursal']= data['Sucursal'].astype(int)
    except:
        print(f"-> Datos No Encontrados en CACHE: {id_proveedor}, Label: {etiqueta}_Solicitudes_Compra")
    return data

In [47]:
import getpass
import uuid
import psycopg2  # Asegurar que tienes instalado psycopg2 para manejar PostgreSQL
from datetime import datetime

def id_aleatorio():
    return str(uuid.uuid4())

def put_execute_result(df_forecast, process_id):
    """
    Inserta los datos de predicción de demanda en la base de datos PostgreSQL.

    :param df_forecast: DataFrame con las predicciones.
    :param process_id: ID del proceso asociado a la ejecución.
    :return: True si la inserción es exitosa, False en caso de error.
    """

    ## GUARDAR TABLA EN POSTGRES
    usuario = getpass.getuser()  # Obtiene el usuario del sistema operativo
    fecha_actual = datetime.today()  # Se mantiene como objeto datetime
    error_margin = 0.05
    confidence_level = 99.4
    lower_bound = 1.0
    upper_bound = 100.0

    # Obtener conexión a PostgreSQL
    try:
        conn = Open_Conn_Connexa()  # Asegúrate de que esta función retorna una conexión válida
    except Exception as e:
        print(f"❌ Error al conectar a la base de datos: {e}")
        return False

    # Query de inserción corregido
    insert_query = """
    INSERT INTO public.spl_supply_forecast_execution_execute_result (
        id, confidence_level, error_margin, expected_demand, average_daily_demand,
        lower_bound, "timestamp", upper_bound, product_id, site_id, supply_forecast_execution_execute_id
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """

    # Convertir el DataFrame a una lista de tuplas para la inserción en bloque
    data_to_insert = [
        (id_aleatorio(), confidence_level, error_margin, row['Forecast'], row['Average'], 
         lower_bound, fecha_actual, upper_bound, row['id_product'], row['id_store'], process_id)
        for _, row in df_forecast.iterrows()
    ]

    # Insertar datos en la base de datos
    try:
        with conn.cursor() as cur:
            cur.executemany(insert_query, data_to_insert)
        conn.commit()
        print(f"✅ Inserción completada: {len(data_to_insert)} registros insertados.")
        return True
    except Exception as e:
        conn.rollback()
        print(f"❌ Error en la inserción: {e}")
        return False
    finally:
        Close_Connection(conn)  # Asegúrate de que esta función cierra la conexión correctamente
        print("✅ Conexión cerrada.")
 

In [ ]:
### Preparar Datos Necesarios

import re
def extract_id_proveedor(name):
    match = re.match(r'^(\d+)_', name)
    return match.group(1) if match else None

Procesos = get_forecast_execution()
# Aplicar la extracción al DataFrame
Procesos['id_proveedor'] = Procesos['name'].apply(extract_id_proveedor)

Productos = get_fnd_product()
Productos['sku']= Productos['sku'].astype(int)
Stores = get_fnd_site()
# Convertir 'code' a int, eliminando los registros que no puedan ser convertidos
stores_cleaned = Stores[pd.to_numeric(Stores['code'], errors='coerce').notna()].copy()
stores_cleaned['code'] = stores_cleaned['code'].astype(int)


-> Generando datos Forecast Excecution
---> Datos de Forecast Execution guardados: data/forecast_exec.csv
-> Generando datos Productos Connexa
---> Datos de PRODUCT en CONNEXA guardados: data/fnd_productc.csv
-> Generando datos Productos Connexa
---> Datos de STORES en CONNEXA guardados: data/fnd_site.csv


In [ ]:
for index, row in Procesos.iterrows():
    print(f"Procesando solicitud para ID: {row['id']}, Nombre: {row['name']} id_Proveedor: {row['id_proveedor']}")
    datos = get_solicitudes(row['id_provedor'], row['name'])

    print ("-----")


In [51]:
id_proveedor = '98'
name= '98_FRATELLI_BRANCA_ALGO_05'
process_id ='747e5b07-82ab-4c4a-8dd6-17b1d20d30f2'

datos = get_solicitudes(id_proveedor, name)


# Realizar el join para obtener el ID del producto en datos
datos_con_id = datos.merge(Productos[['sku', 'id']], left_on='Codigo_Articulo', right_on='sku', how='left')
# Renombrar la columna 'id' a 'id_product'
datos_con_id.rename(columns={'id': 'id_product'}, inplace=True)

# Realizar el join para obtener el ID del producto en datos
datos_con_id = datos_con_id.merge(stores_cleaned[['code', 'id']], left_on='Sucursal', right_on='code', how='left')
# Renombrar la columna 'id' a 'id_store'
datos_con_id.rename(columns={'id': 'id_store'}, inplace=True)


# Eliminar la columna duplicada 'sku' y 'code' que no se necesita
datos_con_id.drop(columns=['sku'], inplace=True)
datos_con_id.drop(columns=['code'], inplace=True)

# Elimino por ahora Filas con campos nulos.
datos_con_id = datos_con_id.dropna()

# Mostrar el DataFrame final con la columna 'id' añadida
#import ace_tools as tools
tools.display_dataframe_to_user(name="Datos con ID", dataframe=datos_con_id)


resultado = put_execute_result(datos_con_id, process_id)

print(resultado)

Datos con ID


✅ Inserción completada: 680 registros insertados.
✅ Conexión cerrada.
True


In [37]:
datos_con_id.info()

<class 'pandas.core.frame.DataFrame'>
Index: 621 entries, 0 to 647
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Codigo_Articulo  621 non-null    int32  
 1   Sucursal         621 non-null    int32  
 2   Forecast         621 non-null    float64
 3   Average          621 non-null    float64
 4   id_product       621 non-null    object 
 5   code             621 non-null    float64
 6   id_store         621 non-null    object 
dtypes: float64(3), int32(2), object(2)
memory usage: 34.0+ KB


In [28]:
Productos['sku']= Productos['sku'].astype(int)
Productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14788 entries, 0 to 14787
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           14788 non-null  object        
 1   description  14788 non-null  object        
 2   ext_code     14788 non-null  object        
 3   sku          14788 non-null  int32         
 4   timestamp    14788 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(1), object(3)
memory usage: 520.0+ KB


In [34]:
# Convertir 'code' a int, eliminando los registros que no puedan ser convertidos
stores_cleaned = Stores[pd.to_numeric(Stores['code'], errors='coerce').notna()].copy()
stores_cleaned['code'] = stores_cleaned['code'].astype(int)

Stores.info()
stores_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          109 non-null    object        
 1   name        109 non-null    object        
 2   timestamp   109 non-null    datetime64[ns]
 3   type        109 non-null    object        
 4   code        109 non-null    object        
 5   company_id  109 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 5.2+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 99 entries, 1 to 108
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          99 non-null     object        
 1   name        99 non-null     object        
 2   timestamp   99 non-null     datetime64[ns]
 3   type        99 non-null     object        
 4   code        99 non-null     int32         
 5   company_id  99

In [ ]:
# Ingresar FORECAST_EXECUTION_SCHEDULE

conn =Open_Conn_Connexa()
cur = conn.cursor()
query = """
-- Insertar registros generando el UUID en la sentencia
INSERT INTO public.spl_supply_forecast_execution_schedule(
	id, "timestamp", supply_forecast_execution_id)

	SELECT
		uuid_generate_v4(),      -- Genera un UUID por cada fila
	    fe."timestamp",          -- Toma el valor del timestamp de la tabla Forecast_Execution (o puede usarse now() )
	    fe.id AS supply_forecast_execution_id
	FROM public.spl_supply_forecast_execution fe;
"""
cur.execute(query)
conn.commit()
cur.close()

SyntaxError: syntax error at or near "FROM"
LINE 11: FROM public.spl_supply_forecast_execution fe;
         ^


In [ ]:
# Ingresar FORECAST_EXECUTION_EXECUTE

conn =Open_Conn_Connexa()
cur = conn.cursor()
query = """
-- Insertar registros generando el UUID en la sentencia
INSERT INTO public.spl_supply_forecast_execution_execute(
	id, end_execution, last_execution, start_execution, "timestamp", 
    supply_forecast_execution_id, supply_forecast_execution_schedule_id
)
SELECT
	uuid_generate_v4(),      -- Genera un UUID por cada fila	    
    fe."timestamp",          -- end_execution
    TRUE,                    -- last_execution (valor booleano)
    fe."timestamp",          -- start_execution
    fe."timestamp",          -- timestamp
	fe.id AS supply_forecast_execution_id,
    fes.id AS supply_forecast_execution_schedule_id
FROM public.spl_supply_forecast_execution fe
JOIN public.spl_supply_forecast_execution_schedule fes
ON fe.id = fes.supply_forecast_execution_id;
"""

cur.execute(query)
conn.commit()
cur.close()